#Importing Dependencies

In [1]:
#primary packages
import pandas as pd
import numpy as np
import warnings
from pprint import pprint

import os
import zipfile
from google.colab import files

#surpress warnings for resmpling and rolling functions
warnings.filterwarnings("ignore", category=UserWarning)


#Reading Data

Upload the file "Data_files.zip" from your file system. The rest is handled automatically by the below block of code. This code will extract the zip folder and add uit to your file system in collab.  I have created this to ease checking of my code on your side . Kindly upload the zip file directly int  the same format you have provided on my email.

In [2]:
uploaded = files.upload()
zip_file = 'Data_files.zip'
output_folder = 'Data_Aamod'

#extract zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

print("Folder unzipped successfully!")

#Path in collab's file system
folder_path = 'Data_Aamod/SampleFiles 2'

#csv files read into dataframes
daily_data = pd.read_csv(folder_path + "/SampleDayData.csv")
april_19 = pd.read_csv(folder_path + "/19thAprilSampleData.csv")
april_22 = pd.read_csv(folder_path + "/22ndAprilSampleData.csv")


Saving Data_files.zip to Data_files.zip
Folder unzipped successfully!


## Checking for empty values

Below code extracts rows with NaN values for each of the dataframes. The result for this data shows that there are no missing/ NaN values. It is wise to do this before defining the fuctions as data with missing values might require data cleaning and robust functions.

In [3]:
empty_indices_daily = np.unique(np.where(daily_data.isnull())[0])
empty_indices_intraday1 = np.unique(np.where(april_19.isnull())[0])
empty_indices_intraday2 = np.unique(np.where(april_22.isnull())[0])
print(len(empty_indices_daily),len(empty_indices_intraday1) ,len(empty_indices_intraday2))

0 0 0


## Fetching all ticker/symbol names

The ticker/ symbol names in daily_data nd intraday_data needs to be i sync. Else data for some of the tickers might need to be dropped. For this case we only need to deal with one ticker that is "ABC".

In [4]:
#fetch all stock names for iteration
stock_names = daily_data["Stock Name"].unique()

#Check numbe rof tickers/symbols
if( len(april_22["Stock Name"].unique()) == len(april_19["Stock Name"].unique()) == len(stock_names)):
  print("DataFrames have same number of tickers")
else:
  print("Warning number of ticker don't match!!!")

DataFrames have same number of tickers


#Function Defintion

Thhe functions "calculate_30_day_average_for_stock" and "calculate_cumulative_volume_for_stock" are helper functions for the function "find_crossovers". The averages are calculated using rolling means with a minimum window size of 1 (ensures non NaN values for smaller window size). Unnecessary columns are eliminated in each of the functions before returning the processed DataFrame, this has been done to keep the data lightweight and reduce processing time of vector operations used in the main function i.e. "find crossovers".



In [5]:
def calculate_30_day_average(daily_data, stock_symbol):
    """
    Calculate the 30-day average volume for a specific stock symbol.
    """
    stock_data = daily_data[daily_data['Stock Name'] == stock_symbol]
    avg_volume = stock_data['Volume'].rolling(window=30, min_periods=1).mean() #min_period =1 ensures non-nan values for first few rows
    stock_data['30_day_avg'] = avg_volume

    #eliminate unnecesary columns
    stock_data = stock_data[["Date","30_day_avg"]] #only return necessary columns
    return stock_data

def calculate_cumulative_volume(intraday_data, stock_symbol, market_open="09:15:00"):
    """
    Calculate cumulative traded volume for a specific stock symbol within a rolling 60-minute window.
    """
    stock_data = intraday_data[intraday_data['Stock Name'] == stock_symbol].copy()

    # Combine Date and Time into a datetime column
    stock_data['datetime'] = pd.to_datetime(stock_data['Date'] + ' ' + stock_data['Time'])

    # Calculate cumulative volume using a rolling 60-minute window
    stock_data.set_index('datetime', inplace=True)

    #eliminate unnecesary columns
    stock_data['cumulative_volume'] = stock_data['Last Traded Quantity'].rolling(pd.Timedelta(minutes=60)).sum()
    stock_data.reset_index(inplace=True)
    return stock_data

def find_crossovers(daily_data, intraday_data, stock_symbol):
    """
    Find the timestamp for a specific stock symbol when cumulative traded volume
    first exceeds the 30-day average volume within a 60-minute rolling window.
    """
    # Get 30-day average data for the stock
    stock_daily_data = calculate_30_day_average(daily_data, stock_symbol)

    # Get intraday data with cumulative volume for the stock
    stock_intraday_data = calculate_cumulative_volume(intraday_data, stock_symbol)

    # Merge the 30-day average into the intraday data
    stock_intraday_data = stock_intraday_data.merge(
        stock_daily_data[['Date', '30_day_avg']],
        on='Date',
        how='left'
    )

    # Find the first timestamp where cumulative volume exceeds the 30-day average
    crossover = stock_intraday_data[stock_intraday_data['cumulative_volume'] > stock_intraday_data['30_day_avg']]
    if not crossover.empty:
        return crossover.iloc[0]['datetime']
    else:
        return None


# Final Result (Crossovers)

Prints 2 dictionaries which contain timestamps for crossovers on each day

In [6]:
# Results for 19th and 22nd April
results_19th = {}
results_22nd = {}

# Loop through each stock and process data
for stock in stock_names:
    results_19th[stock] = find_crossovers(daily_data, april_19, stock)
    results_22nd[stock] = find_crossovers(daily_data, april_22, stock)

# Output results
print("Results for 19th April:")
pprint(results_19th, sort_dicts=False)

print("\nResults for 22nd April:")
pprint(results_22nd, sort_dicts=False)


Results for 19th April:
{'ABC': None}

Results for 22nd April:
{'ABC': Timestamp('2024-04-22 09:41:44')}


The results are stored in the csv files i have shared in the mail as well as on github .

In JSON format Results for 19th April: {'ABC': None}

Results for 22nd April: {'ABC': Timestamp('2024-04-22 09:41:44')}

These are the instances for each day where the volume FIRST crossed the rolling window average as requested in the mail .There can be multiple crosses in a day . ON 19th there wasnt a single cross where as on 22nd there was atleast one crossover .

